In [1]:
from IEMOCAP_reader import IEMOCAPReader
from dataframe_processing import Relabel, Filter, LabelEncoder, OutputMerger
from feature_extractor import Wav2Vec2Embeddings, OpensmileExtractor, Spectrogram
# from splits import Split, GroupSplit, LeaveOneOut
from normalize import NormalizationStatistics
# from generators import BatchGenerator
# from dienen_model import DienenModel
from data_processors import PadDP, ToNumpyDP, SqueezeDP, NormalizeDP, LoadDataframeDP, DownsampleDP
import pandas as pd
import pickle
import numpy as np
import os

In [3]:
data = pd.read_csv('/Users/agnieszkalenart/Documents/mannheim/master_thesis/iemocap_with_features_full.csv')

In [5]:
data

,Unnamed: 0,filename,wavfile,modality,modality_number,modality_code,subject,session,start,end,...,syl_avg_rate,syl_times,annotations_entropy,annotations_entropy_self_report,annotations_full_agreement,sentences,classID,wav2vec2,opensmile,spectrogram
0,Ses01F_impro01_F000,/Users/agnieszkalenart/Documents/mannheim/mast...,Ses01F_impro01_F000.wav,impro,1,impro01,Ses01F,1,0,1.945563,...,5.172414,"[(0.45, 0.5), (0.51, 0.79), (0.8, 1.05)]",0.000000,0.000000,True,EXCUSE ME,2,"[[[0.37165045738220215, 0.3103700876235962, 0....","[[0.4211056, 0.4759358, 0.2032086, 0.1179737, ...","[[0.08872538059949875, 0.08331265300512314, 0...."
1,Ses01F_impro01_F001,/Users/agnieszkalenart/Documents/mannheim/mast...,Ses01F_impro01_F001.wav,impro,1,impro01,Ses01F,1,0,1.382437,...,2.702703,"[(0.53, 0.9)]",0.000000,0.000000,True,YEAH,2,"[[[0.18848827481269836, 0.3724874258041382, 0....","[[0.9830519, 0.4961832, 0.3816794, 0.09562548,...","[[0.011417585425078869, 0.030532516539096832, ..."
2,Ses01F_impro01_F002,/Users/agnieszkalenart/Documents/mannheim/mast...,Ses01F_impro01_F002.wav,impro,1,impro01,Ses01F,1,0,3.130250,...,7.352941,"[(2.1, 2.15), (2.16, 2.28), (2.29, 2.32), (2.3...",0.636514,0.636514,False,IS THERE A PROBLEM,2,"[[[0.16338609158992767, 0.3216764032840729, -0...","[[0.8850114, 0.8039216, 0.0, 0.09186696, 0.104...","[[0.21206150949001312, 0.20897573232650757, 0...."
3,Ses01F_impro01_F005,/Users/agnieszkalenart/Documents/mannheim/mast...,Ses01F_impro01_F005.wav,impro,1,impro01,Ses01F,1,0,4.029937,...,4.347826,"[(0.58, 0.83), (0.84, 1.19), (1.55, 1.7), (1.9...",0.636514,0.636514,False,WELL WHAT'S THE PROBLEM LET ME CHANGE IT,2,"[[[0.29450708627700806, 0.09270311892032623, -...","[[2.740377, 0.4126582, 0.1367089, 0.1386447, 0...","[[0.025045394897460938, 0.03045099787414074, 0..."
4,Ses01F_impro01_F012,/Users/agnieszkalenart/Documents/mannheim/mast...,Ses01F_impro01_F012.wav,impro,1,impro01,Ses01F,1,0,2.750000,...,4.464286,"[(0.36, 0.65), (0.66, 0.85), (0.86, 0.95), (0....",0.636514,0.636514,False,THAT'S OUT OF CONTROL,0,"[[[0.4138171374797821, 0.23242679238319397, 0....","[[4.135432, 0.1828358, 0.9738806, 0.1455787, 0...","[[0.23487713932991028, 0.22447951138019562, 0...."


In [4]:
import ast

#data['wav2vec2'] = data['wav2vec2'].apply(lambda x: ast.literal_eval(x))
#data['wav2vec2'] = data['wav2vec2'].apply(lambda x: x[0])
data['opensmile'] = data['opensmile'].apply(lambda x: ast.literal_eval(x))
#data['spectrogram'] = data['spectrogram'].apply(lambda x: ast.literal_eval(x))


In [5]:
data['opensmile'] = data['opensmile'].apply(np.array)

In [6]:
data['opensmile'] = data['opensmile'].apply(lambda x: x.reshape(6373,))

In [7]:
data['opensmile'][0].shape

(6373,)

In [8]:
data['Unnamed: 0']

0          Ses01F_impro01_F000
1          Ses01F_impro01_F001
2          Ses01F_impro01_F002
3          Ses01F_impro01_F005
4          Ses01F_impro01_F012
                 ...          
1054    Ses01M_script03_2_M039
1055    Ses01M_script03_2_M040
1056    Ses01M_script03_2_M041
1057    Ses01M_script03_2_M042
1058    Ses01M_script03_2_M043
Name: Unnamed: 0, Length: 1059, dtype: object

In [9]:
opensmile_dict = dict(zip(data['Unnamed: 0'], data['opensmile']))


In [10]:
# Save the dictionary as a pickle file
with open('cmn_audio_opensmile.pickle', 'wb') as file:
    pickle.dump(opensmile_dict, file)

In [54]:
list_features = ['wav2vec2', 'opensmile', 'spectrogram']
for feature in list_features:
    for i in range(5):
        print('length of ',feature,' row ', i, 'is ',len(data[feature][i]), ' x ', len(data[feature][i][0]))

length of  wav2vec2  row  0 is  97  x  768
length of  wav2vec2  row  1 is  68  x  768
length of  wav2vec2  row  2 is  156  x  768
length of  wav2vec2  row  3 is  201  x  768
length of  wav2vec2  row  4 is  137  x  768
length of  opensmile  row  0 is  1  x  6373
length of  opensmile  row  1 is  1  x  6373
length of  opensmile  row  2 is  1  x  6373
length of  opensmile  row  3 is  1  x  6373
length of  opensmile  row  4 is  1  x  6373
length of  spectrogram  row  0 is  312  x  201
length of  spectrogram  row  1 is  222  x  201
length of  spectrogram  row  2 is  501  x  201
length of  spectrogram  row  3 is  645  x  201
length of  spectrogram  row  4 is  441  x  201


In [55]:
data['wav2vec2'] = data['wav2vec2'].apply(np.array)
data['opensmile'] = data['opensmile'].apply(np.array)

In [56]:
print(data['opensmile'])

0    [[0.4211056, 0.4759358, 0.2032086, 0.1179737, ...
1    [[0.9830519, 0.4961832, 0.3816794, 0.09562548,...
2    [[0.8850114, 0.8039216, 0.0, 0.09186696, 0.104...
3    [[2.740377, 0.4126582, 0.1367089, 0.1386447, 0...
4    [[4.135432, 0.1828358, 0.9738806, 0.1455787, 0...
Name: opensmile, dtype: object


In [57]:
downsampleDP = DownsampleDP()
data = downsampleDP.process(data=data, factor = 2, col_in='opensmile',col_out='opensmile', axis = 1)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:85: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  y[slc] = x


In [58]:
padDP = PadDP()
data = padDP.process(data=data, col_in = 'opensmile', col_out = 'opensmile', max_length=400)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = x[slc]


In [59]:
#axis = 0?
padDP = PadDP()
data = padDP.process(data=data, col_in = 'wav2vec2', col_out = 'wav2vec2', max_length=400, axis =0)

/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/ser_wav2vec_reimplement/data_processors.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[slc] = x[slc]


In [60]:
squeezeDP = SqueezeDP()
data = squeezeDP.process(data=data, col_in = 'wav2vec2', col_out = 'wav2vec2')

In [61]:
normalization_statistics = NormalizationStatistics()
statistics = normalization_statistics.process(data=data, normalization_by='subject', column ='wav2vec2')
normalizer = NormalizeDP()
data = normalizer.process(data=data, column_in = 'wav2vec2', column_out = 'wav2vec2', statistics=statistics)

In [69]:
print(data['opensmile'])

0    [[0.4485207, 0.16059115, 0.1616628, 0.03528049...
1    [[0.73961755, 0.23865244, 0.12512985, 0.024610...
2    [[0.8444665, 0.04593348, 0.14516535, 0.0470593...
3    [[1.5765176, 0.1376768, 0.69759115, 0.37484245...
4    [[2.1591339, 0.55972965, 0.61022675, 0.3419897...
Name: opensmile, dtype: object


In [62]:
toNumpyDP = ToNumpyDP()
opensmile_data = toNumpyDP.process(data=data, col_in = 'opensmile')

In [63]:
toNumpyDP = ToNumpyDP()
wav2vec2_data = toNumpyDP.process(data=data, col_in = 'wav2vec2')

In [66]:
opensmile_data.shape

(5, 400, 3187)

In [67]:
wav2vec2_data.shape

(5, 400, 768)

In [78]:
opensmile_data[0][0][0]

0.4485207

In [16]:
import arff
import os
TMPARFFPATH = "/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/tmp/tmp.arff"


In [17]:
data = arff.load(open(TMPARFFPATH, 'r'))
print(data['data'])

[['unknown', 0.4211056, 0.4759358, 0.2032086, 0.1179737, 0.1347909, 0.1885347, 0.01681723, 0.05374375, 0.07056098, 0.09733298, 0.5063254, 0.4089924, 0.09974337, 2.021333, 6.238541, 0.22, 0.67, 0.05, 0.2548529, 0.2406417, 0.1069519, 0.06417112, 0.03743315, 0.4247312, 0.5243243, 0.0004625406, -1.484635, 0.4168662, 0.1583346, -0.1107965, 0.03218116, 1.861224, 0.3796791, 0.5775401, 0.8340849, 0.9780678, 1.160645, 0.1439829, 0.1825774, 0.3265603, 0.6233002, 2.3355, 1.712199, 0.3573072, 1.570857, 5.583004, 0.1609091, 0.54, 0.04, 0.166212, 0.3315508, 0.1176471, 0.02139037, 0.01604278, 0.516129, 0.6216216, 0.01260227, -1.358078, 0.2582861, 0.1357087, 0.002304379, -0.02961385, 0.03975631, 0.3850268, 0.2192513, 0.002503606, 0.002860716, 0.004547893, 0.0003571103, 0.001687177, 0.002044287, 0.001869933, 0.04073338, 0.03886345, 0.01064767, 2.172249, 6.193054, 0.2, 0.69, 0.04, 0.2458455, 0.1604278, 0.1229947, 0.0855615, 0.04812834, 0.4892473, 0.5081081, 9.059018e-07, -2.301733, 1.981482, -0.7958212,

In [18]:
filename = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/IEMOCAP/Session1/sentences/wav/Ses01F_impro01/Ses01F_impro01_F000.wav'

In [19]:
MODEL_PATH = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/wav2vec2/wav2vec_small_960h.pt'

CONF_PATH = "/Users/agnieszkalenart/Downloads/opensmile-3.0.0/config/is09-13/IS13_ComParE.conf"
TMPARFFPATH = "/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/tmp/tmp.arff"


In [20]:
os.system("SMILExtract -C " + CONF_PATH + " -I " + filename + " -O " + TMPARFFPATH)
data = arff.load(open(TMPARFFPATH, 'r'))
data_list = []
            #print(type(data['data']))
            #for i in range(len(data['data'])):
data_list.append((data['data'][0][1:-1]))

(MSG) [2] SMILExtract: openSMILE starting!
(MSG) [2] SMILExtract: config file is: /Users/agnieszkalenart/Downloads/opensmile-3.0.0/config/is09-13/IS13_ComParE.conf
(MSG) [2] cComponentManager: successfully registered 103 component types.
(MSG) [2] instance 'lldsink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldhtksink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldarffsink': No filename given, disabling this sink component.
(MSG) [2] instance 'csvsink': No filename given, disabling this sink component.
(MSG) [2] instance 'htksink': No filename given, disabling this sink component.
(MSG) [2] cComponentManager: successfully finished createInstances (48 component instances were finalised, 1 data memories were finalised)
(MSG) [2] cComponentManager: starting single thread processing loop
(WRN) [2] cComponentManager: The following component(s) could not perform any work because destination levels are full but no other component perfor

In [21]:
data_list

[[0.4211056,
  0.4759358,
  0.2032086,
  0.1179737,
  0.1347909,
  0.1885347,
  0.01681723,
  0.05374375,
  0.07056098,
  0.09733298,
  0.5063254,
  0.4089924,
  0.09974337,
  2.021333,
  6.238541,
  0.22,
  0.67,
  0.05,
  0.2548529,
  0.2406417,
  0.1069519,
  0.06417112,
  0.03743315,
  0.4247312,
  0.5243243,
  0.0004625406,
  -1.484635,
  0.4168662,
  0.1583346,
  -0.1107965,
  0.03218116,
  1.861224,
  0.3796791,
  0.5775401,
  0.8340849,
  0.9780678,
  1.160645,
  0.1439829,
  0.1825774,
  0.3265603,
  0.6233002,
  2.3355,
  1.712199,
  0.3573072,
  1.570857,
  5.583004,
  0.1609091,
  0.54,
  0.04,
  0.166212,
  0.3315508,
  0.1176471,
  0.02139037,
  0.01604278,
  0.516129,
  0.6216216,
  0.01260227,
  -1.358078,
  0.2582861,
  0.1357087,
  0.002304379,
  -0.02961385,
  0.03975631,
  0.3850268,
  0.2192513,
  0.002503606,
  0.002860716,
  0.004547893,
  0.0003571103,
  0.001687177,
  0.002044287,
  0.001869933,
  0.04073338,
  0.03886345,
  0.01064767,
  2.172249,
  6.193054,
